In [ ]:
debugging = True
IPTS = '19558'

# Description 

This notebook performs the normalization of a set of images (tiff or fits) by OBs. 

The steps are:
 - select your **sample images**
 - select your **ob images**
 - select your **df images** (optional)
 - all the data will be loaded
 - preview of the sample and ob
 - selection of a **background** region in the sample data
 - normalization is performed
 - export of the normalized data

# Python Import

In [ ]:
from __code.__all import custom_style
custom_style.style()

In [ ]:
from __code.normalization import NormalizationHandler

In [ ]:
%matplotlib notebook

In [ ]:
from __code import utilities, gui_widgets
working_dir = utilities.get_working_dir(ipts=IPTS, debugging=debugging)
print("Working dir: {}".format(working_dir))

# Select Sample Images 

In [ ]:
o_norm_handler = NormalizationHandler(working_dir = working_dir)
o_norm_handler.select_images()

# Select OB 

In [ ]:
o_norm_handler.select_images(data_type='ob')

# Select DF (Optional) 

In [ ]:
o_norm_handler.select_images(data_type='df')

# Loding data 

In [ ]:
o_norm_handler.load_data()

# Preview Data

In [ ]:
o_norm_handler.plot_images(data_type='sample')

In [ ]:
o_norm_handler.plot_images(data_type='ob')

# Select Background Region 

In [ ]:
o_norm_handler.select_sample_roi()

# Normalization

In [ ]:
o_norm_handler.run_normalization()

# Export 

In [ ]:
o_norm_handler.select_export_folder()

In [ ]:
o_norm_handler.export()